![DLI Header](images/DLI_Header.png)

# Star Bikes Marketing Copy Generator

In this notebook you will build an AI-powered marketing copy writer, capable to perform a number of generative tasks. You will learn how to edit the model's **system message** to define its role for response generation.

## Learning Objectives

By the time you complete this notebook you will be able to:
- Perform a variety of **text generation** tasks using LLaMA-2.
- Use **sytem context** to provide the LLaMA-2 model with a definition of its overarching role.

## Video Walkthrough

Execute the cell below to load the video walkthrough of this notebook.

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/04-copy.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## Create LLaMA-2 Pipeline

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## Helper Functions

In this notebook we will use the following functions to support our interaction with the LLM. Feel free to skim over them presently, as they are covered in greater detail when used below.

### Generate Model Responses

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### Costruct Prompt, Optionally With System Context and/or Examples

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## Star Bikes  Data

In [ ]:
bikes = [
    {
        "model": "Galaxy Rider",
        "type": "Mountain",
        "features": {
            "frame": "Aluminum alloy",
            "gears": "21-speed Shimano",
            "brakes": "Hydraulic disc",
            "tires": "27.5-inch all-terrain",
            "suspension": "Full, adjustable",
            "color": "Matte black with green accents"
        },
        "usps": ["Lightweight frame", "Quick gear shift", "Durable tires"],
        "price": 799.95,
        "internal_id": "GR2321",
        "weight": "15.3 kg",
        "manufacturer_location": "Taiwan"
    },
    {
        "model": "Nebula Navigator",
        "type": "Hybrid",
        "features": {
            "frame": "Carbon fiber",
            "gears": "18-speed Nexus",
            "brakes": "Mechanical disc",
            "tires": "26-inch city slick",
            "suspension": "Front only",
            "color": "Glossy white"
        },
        "usps": ["Sleek design", "Efficient on both roads and trails", "Ultra-lightweight"],
        "price": 649.99,
        "internal_id": "NN4120",
        "weight": "13.5 kg",
        "manufacturer_location": "Germany"
    },
    {
        "model": "Cosmic Comet",
        "type": "Road",
        "features": {
            "frame": "Titanium",
            "gears": "24-speed Campagnolo",
            "brakes": "Rim brakes",
            "tires": "700C road",
            "suspension": "None",
            "color": "Metallic blue"
        },
        "usps": ["Super aerodynamic", "High-speed performance", "Professional-grade components"],
        "price": 1199.50,
        "internal_id": "CC5678",
        "weight": "11 kg",
        "manufacturer_location": "Italy"
    }
]

## The Full LLaMA-2 Prompt Template

In the previous notebook, we leveraged the LLaMA-2 **prompt template** to support **few-shot learning**, but mentioned that we were using a slightly modified version of the prompt template. Specifically, we left out a section of the prompt template's user message called the **system message**, or **system context**, or **system prompt** (terms which we will use interchangeably). Below is the entire LLaMA-2 prompt template, including its section for **system context**.

```python
<s>[INST] <<SYS>>
{{ system_context }}
<</SYS>>

{{ user_msg_1 }} [/INST] {{ model_answer_1 }} </s>

```

**System context** is a part of the user side of the user/model interaction, and goes in between the `<<SYS>>` and `<</SYS>>` tags. The **system context** is a preliminary statement or contextual cue designed to orient an AI model's response towards a specific framework or understanding of a task.

There are no hard and fast rules about what belongs in the **system context** but we should consider it primarily to set the role of the model, or any context that will apply all of its responses.

Here is the default **system message** for the LLaMA-2 chat model, used during its instruction fine-tuning:

>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
>If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

## Setting the System Context


The following `construct_prompt_with_context` function will help us constuct prompts with an updated **system message** using the LLaMA-2 **prompt template**. This function also allows us, should we wish, to perform **few-shot learning** by passing in a list of 2-tuple example interactions, as we did in the last section.

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## Star Bikes Marketing Copy Generator

Let's employ our LLaMA-2 model to serve as a marketing copy generator. For its task we will provide it with a JSON object that gives relevant specifications about one of the Star Bikes bicycle models we want it to write copy for. If you haven't already, please check out the *Star Bikes Data* section above for the definition of `bikes`.

We will begin with a simple prompt.

In [ ]:
prompt = f"""
Write marketing copy for the following bicycle: {bikes[0]}
"""

print(generate(prompt))

---

This is not bad, and knowing what you do already, you could probably iterate on a prompt that tightened up the model's response. But assuming we want our model to serve as a marketing copy writer, perhaps writing copy in a variety of formats, let's provide it knowledge about its role by providing it with **system context**.

In [ ]:
system_context = f"""
You are a marketing copy writer for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

Using our `construct_prompt_with_context` function we can now create a prompt that sets the **system context** appropriately for the LLaMA-2 prompt template.

In [ ]:
prompt_with_system_context = construct_prompt_with_context(prompt, system_context)
print(prompt_with_system_context)

---

Using our prompt with **system context**, let's see what kind of response we get back from the model. Worth mentioning is that our main `prompt` (see above) provides no instruction about what the model is to do. We are relying on the set **system context** to guide the model's behavior.

In [ ]:
print(generate(prompt_with_system_context))

---

That's not bad at all, but let's be more **precise** in the **system context** that the model's role is to only generate marketing copy, and not any leading conversation text like `Sure! Here's the marketing copy for the Galaxy Rider mountain bike:` as we got in the last response.

In [ ]:
system_context = f"""
You are a marketing copy writer for Star Bikes. You only write marketing copy and never any \
leading comments or pieces of conversation.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

That did not seem to help. Let's iterate. Perhaps if we tell the model it is a machine, it will not try to have human-like conversations.

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing copy in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

Much better! Just like with all prompt engineering, developing effective **system prompts** is often an iterative process.

## Enforce Brevity

Let's assume that we only want responses that are ~100 words. We can update the **system context** to reflect this.

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing copy in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

Excellent. Now that we have a setup that appears to be working well for us, let's try it out on data from the rest of our bikes:

In [ ]:
for bike in bikes[1:]:
    print(generate(construct_prompt_with_context(bike, system_context)))
    print("\n-----\n")

## Exercise: Generate Marketing Emails

Using what you've learned so far, create a prompt (likely leveraging its **system context**) that writes marketing emails to a user for a specific bike. The email should address the recipient by their name.

See the solution below if you get stuck.

### Your Work Here

### Solution

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing emails in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
Recipient Name: Stella
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

## Key Concept Review

The following key concepts were introduced in this notebook:

- **System Message:** Part of instruction fine-tuned models' prompt templates that allow users to set the role or overarching context of its behavior.

## Optional Advanced Exercises

If you'd like to go above and beyond the requirements of the course, below are some additional open-ended exercises for you to try.

### Use the 7B Model

At the top of the notebook, after restarting the kernel (see cell below), uncomment and use the 7B model instead of the 13B model we demoed. Try to get satisfying results in spite of using the smaller (weaker) model.

### Experiment with What Works Best

We arrived upon several effective prompts using a combination of iterating on a basic prompt, editing the system message, and providing examples (aka "shots") to help the model. It's often more of an art than a science which approach will work better: see if you can get acceptable results by emphasizing changes to all 3 of these "levers".

### Make an Email Generation Pipeline

Expand on the work from the exercise above to create a pipeline that given a collection of recipients, can generate emails for each of them. You might consider creating synthetic user data with more than just the recipient's name, such as bike's they have previously purchased or expressed interest in, whether an email has already been sent to them, etc., and then generating emails that are more relevant to these details.

You might also consider doing more in the context of **few-shot** learning to structure the emails in a specific manner.

## Restart the Kernel

In order to free up GPU memory for the next notebook, please run the following cell to restart the kernel.

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

![DLI Header](images/DLI_Header.png)